<p>
<img align="left" width="200" src="images/peshitta_small.png"/>
<img align="right" width="200" src="images/tf-small.png"/>
</p>

# LinkSyr workshop

## Amsterdam 2018-03-12

### Dirk Roorda

#### dirk.roorda@dans.knaw.nl

<p>
<img align="left" width="200" src="images/etcbc.png"/>
<img align="right" width="200" src="images/dans.png"/>
</p>

# Link Syriaca data

This notebook links Syriaca data to the Syriac New Testament.

We add the links as features to the
[Text-Fabric](https://github.com/Dans-labs/text-fabric) 
representation of SyrNT.

In [1]:
import sys, os, collections
from IPython.display import display, Markdown, HTML
from tf.fabric import Fabric

In [2]:
REPO = '~/github/etcbc/linksyr'
SOURCE = 'syrnt'
CORPUS = f'{REPO}/data/tf/{SOURCE}'

In [3]:
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.2.2
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

37 features found and 0 ignored


# Load Features
We load all available features of the SyrNT data.

In [ ]:
api = TF.load('', silent=True)
allFeatures = TF.explore(silent=True, show=True)
loadableFeatures = allFeatures['nodes'] + allFeatures['edges']
TF.load(loadableFeatures, add=True, silent=True)
api.makeAvailableIn(globals())

In [12]:
print('\n'.join(allFeatures['nodes']))

aspect
book
book@en
chapter
demonstrative_category
feminine_he_dot
gender
grammatical_category
lexeme
lexeme_ascii
noun_type
number
numeral_type
otype
participle_type
person
prefix
prefix_ascii
pronoun_type
root
root_ascii
seyame
state
stem
stem_ascii
suffix
suffix_ascii
suffix_contraction
suffix_gender
suffix_number
suffix_person
verbal_conjugation
verse
word
word_ascii


In [19]:
!head -n 30 {CORPUS}/word.tf

@node
@dataset=syrnt
@datasetName=Syriac New Testament
@description=full form of the word in syriac script
@email1=dirk.roorda@dans.knaw.nl
@encoders=Ancient Biblical Manuscript Center  (transcription),George A. Kiraz and James W. Bennett (database)and Dirk Roorda (TF)
@source=SEDRA
@sourceUrl=https://sedra.bethmardutho.org/about/contributors
@valueType=str
@writtenBy=Text-Fabric
@dateWritten=2018-03-12T07:17:31Z

ܟܬܒܐ
ܕܝܠܝܕܘܬܗ
ܕܝܫܘܥ
ܡܫܝܚܐ
ܒܪܗ
ܕܕܘܝܕ
ܒܪܗ
ܕܐܒܪܗܡ
ܐܒܪܗܡ
ܐܘܠܕ
ܠܐܝܣܚܩ
ܐܝܣܚܩ
ܐܘܠܕ
ܠܝܥܩܘܒ
ܝܥܩܘܒ
ܐܘܠܕ
ܠܝܗܘܕܐ
ܘܠܐܚܘܗܝ


In [17]:
!head -n 30 {CORPUS}/gender.tf

@node
@dataset=syrnt
@datasetName=Syriac New Testament
@description=?
@email1=dirk.roorda@dans.knaw.nl
@encoders=Ancient Biblical Manuscript Center  (transcription),George A. Kiraz and James W. Bennett (database)and Dirk Roorda (TF)
@source=SEDRA
@sourceUrl=https://sedra.bethmardutho.org/about/contributors
@valueType=str
@writtenBy=Text-Fabric
@dateWritten=2018-03-12T07:17:26Z

m
f
n/a
m
m
n/a
m
n/a
n/a
m
n/a
n/a
m
n/a
n/a
m
n/a
m


In [21]:
HTML('''
<style>
.syc {
    font-family: Estrangelo Edessa;
    font-size: 14pt;
}
</style>
''')

## Syriaca data

We load the index of people and places.

In [7]:
SYRIACA = os.path.expanduser(f'{REPO}/data/syriaca')
SC_PEOPLE = f'{SYRIACA}/index_of_persons.csv'
SC_PLACES = f'{SYRIACA}/index_of_places.csv'

SC_URL = 'http://syriaca.org'
SC_PLACE = 'place'
SC_PERSON = 'person'

SC_CONFIG = (
    (SC_PERSON, SC_URL, SC_PEOPLE),
    (SC_PLACE, SC_URL, SC_PLACES),    
)

SC_TYPES = tuple(x[0] for x in SC_CONFIG)

SC_FIELDS = ('trans', 'syriac', 'id')

NA_SYRIAC = {
    '[Syriac Not Available]', 
    '[Syriac Not', 
    '[Syriac',
}

In [14]:
tables = {}
irregular = {}

(transF, syriacF, idF) = SC_FIELDS

for (dataType, baseUrl, dataFile) in SC_CONFIG:
    tables[dataType] = {field: {} for field in SC_FIELDS}
    irregular[dataType] = set()
    dest = tables[dataType]
    irreg = irregular[dataType]
    table = dest[idF]
    indexTrans = dest[transF]
    indexSyriac = dest[syriacF]
    with open(dataFile) as fh:
        for (i, line) in enumerate(fh):
            (transV, syriacV, idV) = line.rstrip('\n').split('\t')
            prefix = f'{baseUrl}/{dataType}/'
            if idV.startswith(prefix):
                idV = idV.replace(prefix, '', 1)
            else:
                irreg.add(idV)
            table[idV] = (transV, syriacV)
            indexTrans.setdefault(transV, set()).add(idV)
            if syriacV not in NA_SYRIAC:
                if '[' in syriacV:
                    print(f'WARNING {dataType} line {i+1}: syriac value "{syriacV}"')
                indexSyriac.setdefault(syriacV, set()).add(idV)

In [15]:
for (dataType, data) in tables.items():
    table = data[idF]
    irreg = irregular[dataType]
    print(f'''
{dataType:>12}s: {len(table):>5} (irregular: {len(irreg):>4})
{"by syriac":>12} : {len(data[syriacF]):>5}
{"by trans":>12} : {len(data[transF]):>5}
''')


      persons:  2371 (irregular:    0)
   by syriac :  1503
    by trans :  1964


       places:  2488 (irregular:    0)
   by syriac :   527
    by trans :  2165



## Link to SyrNT

We can only hope to find connections based on the Syriac.
Let's see if there are words in the SyrNT text that show up in the persons and places lists.

We work with lexemes.

In [17]:
hits = {dataType: {} for dataType in SC_TYPES}

for lx in F.otype.s('lexeme'):
    lex = F.lexeme.v(lx)
    for dataType in SC_TYPES:
        idV = tables[dataType][syriacF].get(lex, None)
        if idV is not None:
            hits[dataType][lx] = idV

In [10]:
for (dataType, theseHits) in hits.items():
    print(f'{dataType:>12}s: {len(theseHits):>5} hits')

      persons:    98 hits
       places:    37 hits


We show the hits by picking the first occurrence of each lexeme and showing it in context.

In [18]:
for (dataType, theseHits) in hits.items():
    markdown = f'''### {dataType}s
lexeme | linked | n-occs | passage | verse text
--- | --- | --- | --- | ---
'''
    for (lx, linked) in sorted(
        theseHits.items(),
        key=lambda x: F.lexeme.v(x[0]),
    ):
        lex = F.lexeme.v(lx)
        ids = ' '.join(sorted(linked))
        occs = L.d(lx, otype='word')
        passage = '{} {}:{}'.format(*T.sectionFromNode(occs[0]))
        verse = L.u(occs[0], otype='verse')[0]
        text = T.text(L.d(verse, otype='word'))
        markdown += (
            f'<span class="syc">{lex}</span> | {ids} | {len(occs)} | {passage} |'
            f' <span class="syc">{text}</syc>\n'
        )
    display(Markdown(markdown))

### persons
lexeme | linked | n-occs | passage | verse text
--- | --- | --- | --- | ---
<span class="syc">ܐܒܐ</span> | 1094 2582 308 | 9 | Matthew 2:22 | <span class="syc">ܟܕ ܕܝܢ ܫܡܥ ܕܐܪܟܠܐܘܣ ܗܘܐ ܡܠܟܐ ܒܝܗܘܕ ܚܠܦ ܗܪܘܕܣ ܐܒܘܗܝ ܕܚܠ ܕܢܐܙܠ ܠܬܡܢ ܘܐܬܚܙܝ ܠܗ ܒܚܠܡܐ ܕܢܐܙܠ ܠܐܬܪܐ ܕܓܠܝܠܐ </syc>
<span class="syc">ܐܒܪܗܡ</span> | 1108 1109 1110 1546 1547 1548 1549 1551 1552 1553 1554 2202 964 | 2 | Matthew 1:1 | <span class="syc">ܟܬܒܐ ܕܝܠܝܕܘܬܗ ܕܝܫܘܥ ܡܫܝܚܐ ܒܪܗ ܕܕܘܝܕ ܒܪܗ ܕܐܒܪܗܡ </syc>
<span class="syc">ܐܕܝ</span> | 1117 1118 2203 | 2 | Luke 3:28 | <span class="syc">ܒܪ ܡܠܟܝ ܒܪ ܐܕܝ ܒܪ ܩܘܣܡ ܒܪ ܐܠܡܘܕܕ ܒܪ ܥܝܪ </syc>
<span class="syc">ܐܕܡ</span> | 1560 | 208 | Luke 3:38 | <span class="syc">ܒܪ ܐܢܘܫ ܒܪ ܫܝܬ ܒܪ ܐܕܡ ܕܡܢ ܐܠܗܐ </syc>
<span class="syc">ܐܗܪܘܢ</span> | 1012 1092 1533 1534 | 3 | Luke 1:5 | <span class="syc">ܗܘܐ ܒܝܘܡܬܗ ܕܗܪܘܕܣ ܡܠܟܐ ܕܝܗܘܕܐ ܟܗܢܐ ܚܕ ܕܫܡܗ ܗܘܐ ܙܟܪܝܐ ܡܢ ܬܫܡܫܬܐ ܕܒܝܬ ܐܒܝܐ ܘܐܢܬܬܗ ܡܢ ܒܢܬܗ ܕܐܗܪܘܢ ܫܡܗ ܗܘܐ ܐܠܝܫܒܥ </syc>
<span class="syc">ܐܘܒܘܠܘܣ</span> | 3028 | 1 | 2_Timothy 4:21 | <span class="syc">ܢܬܒܛܠ ܠܟ ܕܩܕܡ ܣܬܘܐ ܬܐܬܐ ܫܐܠ ܒܫܠܡܟ ܐܘܒܘܠܘܣ ܘܦܘܕܣ ܘܠܝܢܘܣ ܘܩܠܘܕܝܐ ܘܐܚܐ ܟܠܗܘܢ </syc>
<span class="syc">ܐܚܐ</span> | 1122 1123 1740 | 3 | Matthew 1:2 | <span class="syc">ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ </syc>
<span class="syc">ܐܝܣܚܩ</span> | 1788 1789 1790 1791 1792 2578 | 3 | Matthew 1:2 | <span class="syc">ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ </syc>
<span class="syc">ܐܠܝܐ</span> | 1698 1699 1700 1703 1704 1705 2541 3145 945 | 1 | Matthew 2:18 | <span class="syc">ܩܠܐ ܐܫܬܡܥ ܒܪܡܬܐ ܒܟܝܐ ܘܐܠܝܐ ܣܓܝܐܐ ܪܚܝܠ ܒܟܝܐ ܥܠ ܒܢܝܗ ܘܠܐ ܨܒܝܐ ܠܡܬܒܝܐܘ ܡܛܠ ܕܠܐ ܐܝܬܝܗܘܢ </syc>
<span class="syc">ܐܠܟܣܢܕܪܘܣ</span> | 1574 887 | 1 | Mark 15:21 | <span class="syc">ܘܫܚܪܘ ܚܕ ܕܥܒܪ ܗܘܐ ܫܡܥܘܢ ܩܘܪܝܢܝܐ ܕܐܬܐ ܗܘܐ ܡܢ ܩܪܝܬܐ ܐܒܘܗܝ ܕܐܠܟܣܢܕܪܘܣ ܘܕܪܘܦܘܣ ܕܢܫܩܘܠ ܙܩܝܦܗ </syc>
<span class="syc">ܐܡܘܢ</span> | 1579 1580 | 3 | Matthew 1:10 | <span class="syc">ܚܙܩܝܐ ܐܘܠܕ ܠܡܢܫܐ ܡܢܫܐ ܐܘܠܕ ܠܐܡܘܢ ܐܡܘܢ ܐܘܠܕ ܠܝܘܫܝܐ </syc>
<span class="syc">ܐܢܕܪܐܘܣ</span> | 2205 891 954 | 3 | Matthew 4:18 | <span class="syc">ܘܟܕ ܡܗܠܟ ܥܠ ܝܕ ܝܡܐ ܕܓܠܝܠܐ ܚܙܐ ܬܪܝܢ ܐܚܝܢ ܫܡܥܘܢ ܕܐܬܩܪܝ ܟܐܦܐ ܘܐܢܕܪܐܘܣ ܐܚܘܗܝ ܕܪܡܝܢ ܡܨܝܕܬܐ ܒܝܡܐ ܐܝܬܝܗܘܢ ܗܘܘ ܓܝܪ ܨܝܕܐ </syc>
<span class="syc">ܐܢܕܪܘܢܝܩܘܣ</span> | 1588 1589 | 14 | Romans 16:7 | <span class="syc">ܫܐܠܘ ܒܫܠܡܐ ܕܐܢܕܪܘܢܝܩܘܣ ܘܕܝܘܢܝܐ ܐܚܝܢܝ ܕܗܘܘ ܫܒܝܐ ܥܡܝ ܘܝܕܝܥܝܢ ܐܢܘܢ ܒܫܠܝܚܐ ܘܒܡܫܝܚܐ ܩܕܡܝ ܗܘܘ </syc>
<span class="syc">ܐܢܣܝܡܘܣ</span> | 1950 | 18 | Colossians 4:9 | <span class="syc">ܥܡ ܐܢܣܝܡܘܣ ܐܚܐ ܡܗܝܡܢܐ ܘܚܒܝܒܐ ܗܘ ܕܐܝܬܘܗܝ ܡܢܟܘܢ ܗܢܘܢ ܢܘܕܥܘܢܟܘܢ ܡܕܡ ܕܠܘܬܢ </syc>
<span class="syc">ܐܣܛܦܢܐ</span> | 1709 3010 | 55 | 1_Corinthians 1:16 | <span class="syc">ܐܥܡܕܬ ܕܝܢ ܐܦ ܠܒܝܬܗ ܕܐܣܛܦܢܐ ܬܘܒ ܕܝܢ ܠܐ ܝܕܥ ܐܢܐ ܐܢ ܠܐܢܫ ܐܚܪܝܢ ܐܥܡܕܬ </syc>
<span class="syc">ܐܣܛܦܢܘܣ</span> | 1229 1710 | 8 | Acts 6:5 | <span class="syc">ܘܫܦܪܬ ܗܕܐ ܡܠܬܐ ܩܕܡ ܟܠܗ ܥܡܐ ܘܓܒܘ ܠܐܣܛܦܢܘܣ ܓܒܪܐ ܕܡܠܐ ܗܘܐ ܗܝܡܢܘܬܐ ܘܪܘܚܐ ܕܩܘܕܫܐ ܘܠܦܝܠܝܦܘܣ ܘܠܦܪܟܪܘܣ ܘܠܢܝܩܢܘܪ ܘܠܛܝܡܘܢ ܘܠܦܪܡܢܐ ܘܠܢܝܩܠܐܘܣ ܓܝܘܪܐ ܐܢܛܝܘܟܝܐ </syc>
<span class="syc">ܐܣܝܐ</span> | 1616 | 8 | Matthew 9:12 | <span class="syc">ܝܫܘܥ ܕܝܢ ܟܕ ܫܡܥ ܐܡܪ ܠܗܘܢ ܠܐ ܣܢܝܩܝܢ ܚܠܝܡܐ ܥܠ ܐܣܝܐ ܐܠܐ ܐܝܠܝܢ ܕܒܝܫܐܝܬ ܥܒܝܕܝܢ </syc>
<span class="syc">ܐܦܠܘ</span> | 1599 1601 | 20 | Acts 18:24 | <span class="syc">ܘܓܒܪܐ ܚܕ ܕܫܡܗ ܗܘܐ ܐܦܠܘ ܝܗܘܕܝܐ ܕܐܝܬܘܗܝ ܗܘܐ ܛܘܗܡܗ ܡܢ ܐܠܟܣܢܕܪܝܐ ܘܪܕܐ ܗܘܐ ܒܡܠܬܐ ܘܡܕܩ ܗܘܐ ܒܟܬܒܐ ܐܬܐ ܠܐܦܣܘܣ </syc>
<span class="syc">ܐܦܪܝܡ</span> | 1098 13 | 99 | John 11:54 | <span class="syc">ܗܘ ܕܝܢ ܝܫܘܥ ܠܐ ܡܗܠܟ ܗܘܐ ܓܠܝܐܝܬ ܒܝܬ ܝܗܘܕܝܐ ܐܠܐ ܐܙܠ ܠܗ ܡܢ ܬܡܢ ܠܐܬܪܐ ܕܩܪܝܒ ܠܚܘܪܒܐ ܠܟܪܟܐ ܕܡܬܩܪܐ ܐܦܪܝܡ ܘܬܡܢ ܡܬܗܦܟ ܗܘܐ ܥܡ ܬܠܡܝܕܘܗܝ </syc>
<span class="syc">ܐܫܥܝܐ</span> | 1793 2496 3141 | 5 | Matthew 3:3 | <span class="syc">ܗܢܘ ܓܝܪ ܗܘ ܕܐܡܝܪ ܒܝܕ ܐܫܥܝܐ ܢܒܝܐ ܩܠܐ ܕܩܪܐ ܒܚܘܪܒܐ ܛܝܒܘ ܐܘܪܚܗ ܕܡܪܝܐ ܘܐܫܘܘ ܠܫܒܝܠܘܗܝ </syc>
<span class="syc">ܐܬܢܘܣ</span> | 146 | 22 | Acts 17:15 | <span class="syc">ܘܗܢܘܢ ܕܐܬܠܘܝܘ ܠܗ ܠܦܘܠܘܣ ܐܬܘ ܥܡܗ ܥܕܡܐ ܠܐܬܢܘܣ ܡܕܝܢܬܐ ܘܟܕ ܢܦܩܝܢ ܡܢ ܨܐܕܘܗܝ ܩܒܠܘ ܡܢܗ ܐܓܪܬܐ ܠܘܬ ܫܝܠܐ ܘܛܝܡܬܐܘܣ ܕܒܥܓܠ ܢܐܙܠܘܢ ܠܘܬܗ </syc>
<span class="syc">ܒܢܝܡܝܢ</span> | 1172 1632 | 245 | Acts 13:21 | <span class="syc">ܘܗܝܕܝܢ ܫܐܠܘ ܠܗܘܢ ܡܠܟܐ ܘܝܗܒ ܠܗܘܢ ܐܠܗܐ ܠܫܐܘܠ ܒܪ ܩܝܫ ܓܒܪܐ ܡܢ ܫܪܒܬܐ ܕܒܢܝܡܝܢ ܫܢܝܢ ܐܪܒܥܝܢ </syc>
<span class="syc">ܒܪܢܒܐ</span> | 2616 | 9 | Acts 4:36 | <span class="syc">ܝܘܣܦ ܕܝܢ ܗܘ ܕܐܬܟܢܝ ܒܪܢܒܐ ܡܢ ܫܠܝܚܐ ܕܡܬܬܪܓܡ ܒܪܐ ܕܒܘܝܐܐ ܠܘܝܐ ܡܢ ܐܬܪܐ ܕܩܘܦܪܣ </syc>
<span class="syc">ܒܪܢܝܩܐ</span> | 2618 | 65 | Acts 25:13 | <span class="syc">ܘܟܕ ܗܘܘ ܝܘܡܬܐ ܢܚܬ ܐܓܪܦܘܣ ܡܠܟܐ ܘܒܪܢܝܩܐ ܠܩܣܪܝܐ ܕܢܫܐܠܘܢ ܫܠܡܗ ܕܦܗܣܛܘܣ </syc>
<span class="syc">ܒܬܘܠܐ</span> | 2145 | 37 | Revelation 14:4 | <span class="syc">ܗܠܝܢ ܐܢܘܢ ܐܝܠܝܢ ܕܥܡ ܢܫܐ ܠܐ ܐܬܛܘܫܘ ܒܬܘܠܐ ܓܝܪ ܐܝܬܝܗܘܢ ܗܠܝܢ ܕܢܩܦܘܗܝ ܠܐܡܪܐ ܟܠ ܟܪ ܕܢܐܙܠ ܗܠܝܢ ܐܙܕܒܢܘ ܡܢ ܐܢܫܐ ܪܫܝܬܐ ܠܐܠܗܐ ܘܠܐܡܪܐ </syc>
<span class="syc">ܕܕܐ</span> | 1667 | 56 | Colossians 4:10 | <span class="syc">ܫܐܠ ܒܫܠܡܟܘܢ ܐܪܣܛܪܟܘܣ ܫܒܝܐ ܕܥܡܝ ܘܡܪܩܘܣ ܒܪ ܕܕܗ ܕܒܪܢܒܐ ܗܘ ܕܐܬܦܩܕܬܘܢ ܥܠܘܗܝ ܕܐܢ ܐܬܐ ܠܘܬܟܘܢ ܬܩܒܠܘܢܝܗܝ </syc>
<span class="syc">ܕܢܚܐ</span> | 160 444 97 | 2 | Luke 1:78 | <span class="syc">ܒܪܚܡܐ ܕܚܢܢܐ ܕܐܠܗܢ ܕܒܗܘܢ ܢܣܥܪܢ ܕܢܚܐ ܡܢ ܪܘܡܐ </syc>
<span class="syc">ܕܢܝܐܝܠ</span> | 1017 1200 1203 1671 1672 1673 | 28 | Matthew 24:15 | <span class="syc">ܡܐ ܕܝܢ ܕܚܙܝܬܘܢ ܐܬܐ ܛܢܦܬܐ ܕܚܘܪܒܐ ܕܐܬܐܡܪ ܒܕܢܝܐܝܠ ܢܒܝܐ ܕܩܝܡܐ ܒܕܘܟܬܐ ܩܕܝܫܬܐ ܗܘ ܕܩܪܐ ܢܣܬܟܠ </syc>
<span class="syc">ܗܒܝܠ</span> | 1540 1541 | 4 | Matthew 23:35 | <span class="syc">ܐܝܟܢܐ ܕܢܐܬܐ ܥܠܝܟܘܢ ܟܠܗ ܕܡܐ ܕܙܕܝܩܐ ܕܐܬܐܫܕ ܥܠ ܐܪܥܐ ܡܢ ܕܡܗ ܕܗܒܝܠ ܙܕܝܩܐ ܘܥܕܡܐ ܠܕܡܗ ܕܙܟܪܝܐ ܒܪ ܒܪܟܝܐ ܗܘ ܕܩܛܠܬܘܢ ܒܝܢܝ ܗܝܟܠܐ ܠܡܕܒܚܐ </syc>
<span class="syc">ܗܘܫܥ</span> | 1952 | 39 | Romans 9:25 | <span class="syc">ܐܝܟܢܐ ܕܐܦ ܒܗܘܫܥ ܐܡܪ ܕܐܩܪܐ ܠܐܝܠܝܢ ܕܠܐ ܗܘܘ ܥܡܝ ܥܡܐ ܕܝܠܝ ܘܠܠܐ ܐܬܪܚܡܬ ܐܬܪܚܡܬ </syc>
<span class="syc">ܙܟܝ</span> | 2156 | 26 | Luke 19:2 | <span class="syc">ܓܒܪܐ ܚܕ ܕܫܡܗ ܙܟܝ ܥܬܝܪܐ ܗܘܐ ܘܪܒ ܡܟܣܐ </syc>
<span class="syc">ܙܟܪܝܐ</span> | 2153 2155 2476 | 2 | Matthew 23:35 | <span class="syc">ܐܝܟܢܐ ܕܢܐܬܐ ܥܠܝܟܘܢ ܟܠܗ ܕܡܐ ܕܙܕܝܩܐ ܕܐܬܐܫܕ ܥܠ ܐܪܥܐ ܡܢ ܕܡܗ ܕܗܒܝܠ ܙܕܝܩܐ ܘܥܕܡܐ ܠܕܡܗ ܕܙܟܪܝܐ ܒܪ ܒܪܟܝܐ ܗܘ ܕܩܛܠܬܘܢ ܒܝܢܝ ܗܝܟܠܐ ܠܡܕܒܚܐ </syc>
<span class="syc">ܙܢܝܬܐ</span> | 1658 | 39 | Matthew 21:31 | <span class="syc">ܡܢܘ ܡܢ ܗܠܝܢ ܬܪܝܗܘܢ ܥܒܕ ܨܒܝܢܐ ܕܐܒܘܗܝ ܐܡܪܝܢ ܠܗ ܗܘ ܩܕܡܝܐ ܐܡܪ ܠܗܘܢ ܝܫܘܥ ܐܡܝܢ ܐܡܪ ܐܢܐ ܠܟܘܢ ܕܡܟܣܐ ܘܙܢܝܬܐ ܩܕܡܝܢ ܠܟܘܢ ܠܡܠܟܘܬܐ ܕܐܠܗܐ </syc>
<span class="syc">ܙܥܘܪܐ</span> | 1524 | 4 | Matthew 5:19 | <span class="syc">ܟܠ ܡܢ ܗܟܝܠ ܕܢܫܪܐ ܚܕ ܡܢ ܦܘܩܕܢܐ ܗܠܝܢ ܙܥܘܪܐ ܘܢܠܦ ܗܟܢܐ ܠܒܢܝܢܫܐ ܒܨܝܪܐ ܢܬܩܪܐ ܒܡܠܟܘܬܐ ܕܫܡܝܐ ܟܠ ܕܝܢ ܕܢܥܒܕ ܘܢܠܦ ܗܢܐ ܪܒܐ ܢܬܩܪܐ ܒܡܠܟܘܬܐ ܕܫܡܝܐ </syc>
<span class="syc">ܚܘܐ</span> | 1731 | 2 | Matthew 2:8 | <span class="syc">ܘܫܕܪ ܐܢܘܢ ܠܒܝܬ-ܠܚܡ ܘܐܡܪ ܠܗܘܢ ܙܠܘ ܥܩܒܘ ܥܠ ܛܠܝܐ ܚܦܝܛܐܝܬ ܘܡܐ ܕܐܫܟܚܬܘܢܝܗܝ ܬܘ ܚܘܐܘܢܝ ܕܐܦ ܐܢܐ ܐܙܠ ܐܣܓܘܕ ܠܗ </syc>
<span class="syc">ܚܠܐ</span> | 1769 | 12 | Matthew 7:26 | <span class="syc">ܘܟܠ ܡܢ ܕܫܡܥ ܡܠܝ ܗܠܝܢ ܘܠܐ ܥܒܕ ܠܗܝܢ ܢܬܕܡܐ ܠܓܒܪܐ ܣܟܠܐ ܕܒܢܐ ܒܝܬܗ ܥܠ ܚܠܐ </syc>
<span class="syc">ܚܠܝܐ</span> | 1031 | 16 | James 3:11 | <span class="syc">ܕܠܡܐ ܡܫܟܚܐ ܕܡܢ ܚܕ ܡܒܘܥܐ ܢܦܩܘܢ ܡܝܐ ܚܠܝܐ ܘܡܪܝܪܐ </syc>
<span class="syc">ܚܢܐ</span> | 1590 | 4 | Luke 2:36 | <span class="syc">ܘܚܢܐ ܕܝܢ ܢܒܝܬܐ ܒܪܬܗ ܕܦܢܘܐܝܠ ܡܢ ܫܒܛܐ ܕܐܫܝܪ ܐܦ ܗܝ ܩܫܝܫܬ ܒܝܘܡܬܗ ܗܘܬ ܘܫܒܥ ܫܢܝܢ ܥܡ ܒܥܠܗ ܚܝܬ ܡܢ ܒܬܘܠܘܬܗ </syc>
<span class="syc">ܚܢܢܝܐ</span> | 1262 1581 1582 | 146 | Acts 5:1 | <span class="syc">ܘܓܒܪܐ ܚܕ ܕܫܡܗ ܗܘܐ ܚܢܢܝܐ ܥܡ ܐܢܬܬܗ ܕܫܡܗ ܗܘܐ ܫܦܝܪܐ ܙܒܢ ܗܘܐ ܩܪܝܬܗ </syc>
<span class="syc">ܚܬܐ</span> | 3030 | 33 | Matthew 12:50 | <span class="syc">ܟܠܢܫ ܓܝܪ ܕܥܒܕ ܨܒܝܢܗ ܕܐܒܝ ܕܒܫܡܝܐ ܗܘܝܘ ܐܚܝ ܘܚܬܝ ܘܐܡܝ </syc>
<span class="syc">ܛܘܒܢܐ</span> | 120 | 25 | John 13:17 | <span class="syc">ܐܢ ܗܠܝܢ ܬܕܥܘܢ ܛܘܒܢܐ ܐܢܬܘܢ ܐܢ ܬܥܒܕܘܢ ܐܢܝܢ </syc>
<span class="syc">ܛܝܡܬܐܘܣ</span> | 2129 2130 2773 | 65 | Acts 16:1 | <span class="syc">ܘܡܛܝ ܗܘܐ ܠܕܪܒܐ ܡܕܝܢܬܐ ܘܠܠܘܣܛܪܐ ܐܝܬ ܗܘܐ ܕܝܢ ܬܡܢ ܬܠܡܝܕܐ ܚܕ ܕܫܡܗ ܛܝܡܬܐܘܣ ܒܪܗ ܕܝܗܘܕܝܬܐ ܚܕܐ ܡܗܝܡܢܬܐ ܘܐܒܘܗܝ ܐܪܡܝܐ ܗܘܐ </syc>
<span class="syc">ܛܠܝܐ</span> | 1494 | 1 | Matthew 2:8 | <span class="syc">ܘܫܕܪ ܐܢܘܢ ܠܒܝܬ-ܠܚܡ ܘܐܡܪ ܠܗܘܢ ܙܠܘ ܥܩܒܘ ܥܠ ܛܠܝܐ ܚܦܝܛܐܝܬ ܘܡܐ ܕܐܫܟܚܬܘܢܝܗܝ ܬܘ ܚܘܐܘܢܝ ܕܐܦ ܐܢܐ ܐܙܠ ܐܣܓܘܕ ܠܗ </syc>
<span class="syc">ܝܗܘܕܐ</span> | 3125 | 1 | Matthew 1:2 | <span class="syc">ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ </syc>
<span class="syc">ܝܘܚܢܢ</span> | 1321 1801 1802 1803 1805 1806 1811 1813 1818 1820 1821 1826 2400 2424 2449 2477 2683 2685 2686 2787 854 977 | 1 | Matthew 3:1 | <span class="syc">ܒܗܘܢ ܕܝܢ ܒܝܘܡܬܐ ܗܢܘܢ ܐܬܐ ܝܘܚܢܢ ܡܥܡܕܢܐ ܘܡܟܪܙ ܗܘܐ ܒܚܘܪܒܐ ܕܝܗܘܕ </syc>
<span class="syc">ܝܘܠܝܘܣ</span> | 2163 | 9 | Acts 27:1 | <span class="syc">ܘܦܩܕ ܥܠܘܗܝ ܦܗܣܛܣ ܕܢܫܬܕܪ ܠܘܬ ܩܣܪ ܠܐܝܛܠܝܐ ܘܐܫܠܡܗ ܠܦܘܠܘܣ ܘܠܐܣܝܪܐ ܐܚܪܢܐ ܥܡܗ ܠܓܒܪܐ ܚܕ ܩܢܛܪܘܢܐ ܡܢ ܐܣܦܝܪ ܣܒܣܛܐ ܕܫܡܗ ܗܘܐ ܝܘܠܝܘܣ </syc>
<span class="syc">ܝܘܢܢ</span> | 1513 1830 | 6 | Matthew 12:39 | <span class="syc">ܗܘ ܕܝܢ ܥܢܐ ܘܐܡܪ ܠܗܘܢ ܫܪܒܬܐ ܒܝܫܬܐ ܘܓܝܪܬܐ ܐܬܐ ܒܥܝܐ ܘܐܬܐ ܠܐ ܬܬܝܗܒ ܠܗ ܐܠܐ ܐܬܗ ܕܝܘܢܢ ܢܒܝܐ </syc>
<span class="syc">ܝܘܣܦ</span> | 1836 1837 1838 2162 2475 | 1 | Matthew 1:16 | <span class="syc">ܝܥܩܘܒ ܐܘܠܕ ܠܝܘܣܦ ܓܒܪܗ ܕܡܪܝܡ ܕܡܢܗ ܐܬܝܠܕ ܝܫܘܥ ܕܡܬܩܪܐ ܡܫܝܚܐ </syc>
<span class="syc">ܝܚܝܕܝܐ</span> | 2088 | 183 | Luke 7:12 | <span class="syc">ܘܟܕ ܩܪܒ ܠܬܪܥܐ ܕܡܕܝܢܬܐ ܚܙܐ ܟܕ ܡܠܘܝܢ ܡܝܬܐ ܕܝܚܝܕܝܐ ܗܘܐ ܠܐܡܗ ܘܗܝ ܐܡܗ ܐܪܡܠܬܐ ܗܘܬ ܘܟܢܫܐ ܣܓܝܐܐ ܕܒܢܝ ܡܕܝܢܬܐ ܥܡܗ </syc>
<span class="syc">ܝܠܕܐ</span> | 1052 | 4 | Matthew 1:18 | <span class="syc">ܝܠܕܗ ܕܝܢ ܕܝܫܘܥ ܡܫܝܚܐ ܗܟܢܐ ܗܘܐ ܟܕ ܡܟܝܪܐ ܗܘܬ ܡܪܝܡ ܐܡܗ ܠܝܘܣܦ ܥܕܠܐ ܢܫܬܘܬܦܘܢ ܐܫܬܟܚܬ ܒܛܢܐ ܡܢ ܪܘܚܐ ܕܩܘܕܫܐ </syc>
<span class="syc">ܝܥܩܘܒ</span> | 1299 1301 1302 1797 1798 1800 962 | 3 | Matthew 1:2 | <span class="syc">ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ </syc>
<span class="syc">ܝܪܬ</span> | 1509 | 5 | Matthew 5:5 | <span class="syc">ܛܘܒܝܗܘܢ ܠܡܟܝܟܐ ܕܗܢܘܢ ܢܐܪܬܘܢ ܠܐܪܥܐ </syc>
<span class="syc">ܟܪܘܒܐ</span> | 1644 | 1 | 1_Corinthians 9:10 | <span class="syc">ܐܠܐ ܝܕܝܥܐ ܕܡܛܠܬܢ ܗܘ ܐܡܪ ܘܡܛܠܬܢ ܓܝܪ ܐܬܟܬܒܬ ܡܛܠ ܕܥܠ ܣܒܪܐ ܗܘ ܘܠܐ ܠܗ ܠܟܪܘܒܐ ܕܢܟܪܘܒ ܘܐܝܢܐ ܕܡܕܪܟ ܥܠ ܣܒܪܐ ܕܥܠܠܬܐ </syc>
<span class="syc">ܠܘܩܐ</span> | 1855 | 3 | Colossians 4:14 | <span class="syc">ܫܐܠ ܒܫܠܡܟܘܢ ܠܘܩܐ ܐܣܝܐ ܚܒܝܒܢ ܘܕܡܐ </syc>
<span class="syc">ܠܥܙܪ</span> | 1852 | 9 | Luke 16:20 | <span class="syc">ܘܡܣܟܢܐ ܚܕ ܐܝܬ ܗܘܐ ܕܫܡܗ ܠܥܙܪ ܘܪܡܐ ܗܘܐ ܠܘܬ ܬܪܥܗ ܕܗܘ ܥܬܝܪܐ ܟܕ ܡܡܚܝ ܒܫܘܚܢܐ </syc>
<span class="syc">ܡܐܢܐ</span> | 2705 | 26 | Matthew 9:21 | <span class="syc">ܐܡܪܐ ܗܘܬ ܓܝܪ ܒܢܦܫܗ ܐܦܢ ܒܠܚܘܕ ܠܡܐܢܗ ܩܪܒܐ ܐܢܐ ܡܬܐܣܝܐ ܐܢܐ </syc>
<span class="syc">ܡܐܢܝ</span> | 608 | 2 | Luke 3:31 | <span class="syc">ܒܪ ܡܠܝܐ ܒܪ ܡܐܢܝ ܒܪ ܡܛܬܐ ܒܪ ܢܬܢ ܒܪ ܕܘܝܕ </syc>
<span class="syc">ܡܝܟܐܝܠ</span> | 1921 637 | 33 | Jude 1:9 | <span class="syc">ܡܝܟܐܝܠ ܕܝܢ ܪܝܫ ܡܠܐܟܐ ܗܘ ܕܥܡ ܐܟܠܩܪܨܐ ܟܕ ܕܐܢ ܡܡܠܠ ܗܘܐ ܡܛܠ ܦܓܪܗ ܕܡܘܫܐ ܠܐ ܐܡܪܚ ܕܢܝܬܐ ܥܠܘܗܝ ܕܝܢܐ ܕܓܘܕܦܐ ܐܠܐ ܐܡܪ ܕܢܓܥܘܪ ܒܟ ܡܪܝܐ </syc>
<span class="syc">ܡܝܠܝܛܘܣ</span> | 1919 | 51 | Acts 20:15 | <span class="syc">ܘܡܢ ܬܡܢ ܠܝܘܡܐ ܐܚܪܢܐ ܪܕܝܢ ܠܘܩܒܠ ܟܝܘܣ ܓܙܪܬܐ ܘܬܘܒ ܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܣܡܘܣ ܘܩܘܝܢ ܒܛܪܘܓܠܝܘܢ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܡܝܠܝܛܘܣ </syc>
<span class="syc">ܡܠܟܐ</span> | 1350 1862 | 3 | Matthew 1:6 | <span class="syc">ܐܝܫܝ ܐܘܠܕ ܠܕܘܝܕ ܡܠܟܐ ܕܘܝܕ ܐܘܠܕ ܠܫܠܝܡܘܢ ܡܢ ܐܢܬܬܗ ܕܐܘܪܝܐ </syc>
<span class="syc">ܡܣܟܢܐ</span> | 1988 | 2 | Matthew 5:3 | <span class="syc">ܛܘܒܝܗܘܢ ܠܡܣܟܢܐ ܒܪܘܚ ܕܕܝܠܗܘܢ ܗܝ ܡܠܟܘܬܐ ܕܫܡܝܐ </syc>
<span class="syc">ܡܪܝܡ</span> | 1880 624 | 4 | Matthew 1:16 | <span class="syc">ܝܥܩܘܒ ܐܘܠܕ ܠܝܘܣܦ ܓܒܪܗ ܕܡܪܝܡ ܕܡܢܗ ܐܬܝܠܕ ܝܫܘܥ ܕܡܬܩܪܐ ܡܫܝܚܐ </syc>
<span class="syc">ܡܪܩܘܣ</span> | 1356 1869 1870 1872 2557 619 | 15 | Acts 12:12 | <span class="syc">ܘܟܕ ܐܣܬܟܠ ܐܬܐ ܠܗ ܠܒܝܬܐ ܕܡܪܝܡ ܐܡܗ ܕܝܘܚܢܢ ܗܘ ܕܐܬܟܢܝ ܡܪܩܘܣ ܡܛܠ ܕܐܚܐ ܣܓܝܐܐ ܬܡܢ ܟܢܝܫܝܢ ܗܘܘ ܘܡܨܠܝܢ </syc>
<span class="syc">ܡܪܬܐ</span> | 1370 | 137 | Luke 10:38 | <span class="syc">ܘܗܘܐ ܕܟܕ ܗܢܘܢ ܪܕܝܢ ܒܐܘܪܚܐ ܥܠ ܠܩܪܝܬܐ ܚܕܐ ܘܐܢܬܬܐ ܕܫܡܗ ܡܪܬܐ ܩܒܠܬܗ ܒܒܝܬܗ </syc>
<span class="syc">ܡܬܝ</span> | 1909 | 13 | Matthew 9:9 | <span class="syc">ܘܟܕ ܥܒܪ ܝܫܘܥ ܡܢ ܬܡܢ ܚܙܐ ܓܒܪܐ ܕܝܬܒ ܒܝܬ ܡܟܣܐ ܕܫܡܗ ܡܬܝ ܘܐܡܪ ܠܗ ܬܐ ܒܬܪܝ ܘܩܡ ܐܙܠ ܒܬܪܗ </syc>
<span class="syc">ܢܘܚ</span> | 1949 3022 | 13 | Matthew 10:15 | <span class="syc">ܘܐܡܝܢ ܐܡܪ ܐܢܐ ܠܟܘܢ ܕܠܐܪܥܐ ܕܣܕܘܡ ܘܕܥܡܘܪܐ ܢܗܘܐ ܢܝܚ ܒܝܘܡܐ ܕܕܝܢܐ ܐܘ ܠܡܕܝܢܬܐ ܗܝ </syc>
<span class="syc">ܢܘܢܐ</span> | 2292 | 3 | Matthew 7:10 | <span class="syc">ܘܐܢ ܢܘܢܐ ܢܫܐܠܝܘܗܝ ܠܡܐ ܚܘܝܐ ܡܘܫܛ ܠܗ </syc>
<span class="syc">ܢܝܩܠܐܘܣ</span> | 1946 | 2 | Acts 6:5 | <span class="syc">ܘܫܦܪܬ ܗܕܐ ܡܠܬܐ ܩܕܡ ܟܠܗ ܥܡܐ ܘܓܒܘ ܠܐܣܛܦܢܘܣ ܓܒܪܐ ܕܡܠܐ ܗܘܐ ܗܝܡܢܘܬܐ ܘܪܘܚܐ ܕܩܘܕܫܐ ܘܠܦܝܠܝܦܘܣ ܘܠܦܪܟܪܘܣ ܘܠܢܝܩܢܘܪ ܘܠܛܝܡܘܢ ܘܠܦܪܡܢܐ ܘܠܢܝܩܠܐܘܣ ܓܝܘܪܐ ܐܢܛܝܘܟܝܐ </syc>
<span class="syc">ܢܬܢܝܐܝܠ</span> | 1942 651 | 116 | John 1:45 | <span class="syc">ܘܦܝܠܝܦܘܣ ܐܫܟܚ ܠܢܬܢܝܐܝܠ ܘܐܡܪ ܠܗ ܗܘ ܕܟܬܒ ܥܠܘܗܝ ܡܘܫܐ ܒܢܡܘܣܐ ܘܒܢܒܝܐ ܐܫܟܚܢܝܗܝ ܕܝܫܘܥ ܗܘ ܒܪ ܝܘܣܦ ܡܢ ܢܨܪܬ </syc>
<span class="syc">ܣܒܐ</span> | 2045 2046 2047 | 11 | Luke 1:18 | <span class="syc">ܘܐܡܪ ܙܟܪܝܐ ܠܡܠܐܟܐ ܐܝܟܢܐ ܐܕܥ ܗܕܐ ܐܢܐ ܓܝܪ ܐܝܬܝ ܣܒܐ ܘܐܢܬܬܝ ܣܓܝܐܬ ܒܝܘܡܬܗ ܗܝ </syc>
<span class="syc">ܣܝܢܝ</span> | 3031 | 54 | Acts 7:30 | <span class="syc">ܘܟܕ ܡܠܝ ܠܗ ܬܡܢ ܐܪܒܥܝܢ ܫܢܝܢ ܐܬܚܙܝ ܠܗ ܒܡܕܒܪܐ ܕܛܘܪ ܣܝܢܝ ܡܠܐܟܗ ܕܡܪܝܐ ܒܢܘܪܐ ܕܝܩܕܐ ܒܣܢܝܐ </syc>
<span class="syc">ܣܩܘܢܕܘܣ</span> | 727 | 2 | Acts 20:4 | <span class="syc">ܘܢܦܩܘ ܥܡܗ ܥܕܡܐ ܠܐܣܝܐ ܣܘܦܛܪܘܣ ܕܡܢ ܒܪܘܐܐ ܡܕܝܢܬܐ ܘܐܪܣܛܪܟܘܣ ܘܣܩܘܢܕܘܣ ܕܡܢ ܬܣܠܘܢܝܩܐ ܘܓܐܝܘܣ ܕܡܢ ܕܪܒܐ ܡܕܝܢܬܐ ܘܛܝܡܬܐܘܣ ܕܡܢ ܠܘܣܛܪܐ ܘܡܢ ܐܣܝܐ ܛܘܟܝܩܘܣ ܘܛܪܘܦܝܡܘܣ </syc>
<span class="syc">ܣܪܐ</span> | 2063 | 65 | John 11:39 | <span class="syc">ܘܐܡܪ ܝܫܘܥ ܫܩܘܠܘ ܟܐܦܐ ܗܕܐ ܐܡܪܐ ܠܗ ܡܪܬܐ ܚܬܗ ܕܗܘ ܡܝܬܐ ܡܪܝ ܡܢ ܟܕܘ ܣܪܝ ܠܗ ܐܪܒܥܐ ܠܗ ܓܝܪ ܝܘܡܝܢ </syc>
<span class="syc">ܥܒܕܐ</span> | 1097 1099 | 2 | Matthew 5:16 | <span class="syc">ܗܟܢܐ ܢܢܗܪ ܢܘܗܪܟܘܢ ܩܕܡ ܒܢܝܢܫܐ ܕܢܚܙܘܢ ܥܒܕܝܟܘܢ ܛܒܐ ܘܢܫܒܚܘܢ ܠܐܒܘܟܘܢ ܕܒܫܡܝܐ </syc>
<span class="syc">ܥܙܝܙܐ</span> | 1531 | 97 | Acts 2:2 | <span class="syc">ܗܘܐ ܡܢ ܫܠܝܐ ܡܢ ܫܡܝܐ ܩܠܐ ܐܝܟ ܪܘܚܐ ܥܙܝܙܬܐ ܘܐܬܡܠܝ ܗܘܐ ܡܢܗ ܟܠܗ ܒܝܬܐ ܗܘ ܕܒܗ ܝܬܒܝܢ ܗܘܘ </syc>
<span class="syc">ܥܡܐܘܣ</span> | 91 | 56 | Luke 24:13 | <span class="syc">ܘܗܐ ܬܪܝܢ ܡܢܗܘܢ ܒܗ ܒܝܘܡܐ ܐܙܠܝܢ ܗܘܘ ܠܩܪܝܬܐ ܕܫܡܗ ܥܡܐܘܣ ܘܦܪܝܩܐ ܡܢ ܐܘܪܫܠܡ ܐܣܛܕܘܬܐ ܫܬܝܢ </syc>
<span class="syc">ܥܡܢܘܐܝܠ</span> | 473 | 1 | Matthew 1:23 | <span class="syc">ܕܗܐ ܒܬܘܠܬܐ ܬܒܛܢ ܘܬܐܠܕ ܒܪܐ ܘܢܩܪܘܢ ܫܡܗ ܥܡܢܘܐܝܠ ܕܡܬܬܪܓܡ ܥܡܢ ܐܠܗܢ </syc>
<span class="syc">ܦܐܛܪܐ</span> | 2013 | 29 | Acts 21:1 | <span class="syc">ܘܦܪܫܢ ܡܢܗܘܢ ܘܪܕܝܢ ܬܪܝܨܐܝܬ ܠܩܘ ܓܙܪܬܐ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܪܘܕܘܣ ܘܡܢ ܬܡܢ ܠܦܐܛܪܐ </syc>
<span class="syc">ܦܘܠܘܣ</span> | 1405 1976 1977 1978 1981 1983 1985 1986 1987 1990 3148 | 69 | Acts 13:9 | <span class="syc">ܫܐܘܠ ܕܝܢ ܗܘ ܕܐܬܩܪܝ ܦܘܠܘܣ ܐܬܡܠܝ ܗܘܐ ܒܪܘܚܐ ܕܩܘܕܫܐ ܘܚܪ ܒܗ </syc>
<span class="syc">ܦܘܦܠܝܘܣ</span> | 888 | 15 | Acts 28:7 | <span class="syc">ܐܝܬ ܗܘܐ ܕܝܢ ܩܘܪܝܐ ܒܗ ܒܗܘ ܐܬܪܐ ܠܓܒܪܐ ܚܕ ܕܫܡܗ ܗܘܐ ܦܘܦܠܝܘܣ ܕܗܘ ܗܘܐ ܪܫܗ ܕܓܙܪܬܐ ܘܩܒܠܢ ܒܒܝܬܗ ܬܠܬܐ ܝܘܡܝܢ ܚܕܝܐܝܬ </syc>
<span class="syc">ܦܛܪܘܣ</span> | 2005 2008 2011 2409 2733 | 19 | Acts 1:13 | <span class="syc">ܘܡܢ ܒܬܪ ܕܥܠܘ ܣܠܩܘ ܠܗܘܢ ܠܥܠܝܬܐ ܗܝ ܕܗܘܝܢ ܗܘܘ ܒܗ ܦܛܪܘܣ ܘܝܘܚܢܢ ܘܝܥܩܘܒ ܘܐܢܕܪܐܘܣ ܘܦܝܠܝܦܘܣ ܘܬܐܘܡܐ ܘܡܬܝ ܘܒܪ-ܬܘܠܡܝ ܘܝܥܩܘܒ ܒܪ ܚܠܦܝ ܘܫܡܥܘܢ ܛܢܢܐ ܘܝܗܘܕܐ ܒܪ ܝܥܩܘܒ </syc>
<span class="syc">ܦܝܠܝܡܘܢ</span> | 1996 | 4 | Philemon 1:1 | <span class="syc">ܦܘܠܘܣ ܐܣܝܪܗ ܕܝܫܘܥ ܡܫܝܚܐ ܘܛܝܡܬܐܘܣ ܐܚܐ ܠܦܝܠܡܘܢ ܚܒܝܒܐ ܘܦܠܚܐ ܕܥܡܢ </syc>
<span class="syc">ܦܝܠܝܦܘܣ</span> | 1998 951 | 1 | Matthew 10:3 | <span class="syc">ܘܦܝܠܝܦܘܣ ܘܒܪ-ܬܘܠܡܝ ܘܬܐܘܡܐ ܘܡܬܝ ܡܟܣܐ ܘܝܥܩܘܒ ܒܪ ܚܠܦܝ ܘܠܒܝ ܕܐܬܟܢܝ ܬܕܝ </syc>
<span class="syc">ܦܦܘܣ</span> | 1965 | 13 | Acts 13:6 | <span class="syc">ܘܟܕ ܐܬܟܪܟܘܗ ܠܟܠܗ ܓܙܪܬܐ ܥܕܡܐ ܠܦܦܘܣ ܡܕܝܢܬܐ ܐܫܟܚܘ ܓܒܪܐ ܚܕ ܚܪܫܐ ܝܗܘܕܝܐ ܕܐܝܬܘܗܝ ܗܘܐ ܢܒܝܐ ܕܓܠܐ ܕܫܡܗ ܗܘܐ ܒܪܫܘܡܐ </syc>
<span class="syc">ܨܠܝܒܐ</span> | 1661 | 10 | Mark 10:21 | <span class="syc">ܝܫܘܥ ܕܝܢ ܚܪ ܒܗ ܘܐܚܒܗ ܘܐܡܪ ܠܗ ܚܕܐ ܚܣܝܪܐ ܠܟ ܙܠ ܙܒܢ ܟܠ ܡܕܡ ܕܐܝܬ ܠܟ ܘܗܒ ܠܡܣܟܢܐ ܘܬܗܘܐ ܠܟ ܣܝܡܬܐ ܒܫܡܝܐ ܘܣܒ ܨܠܝܒܐ ܘܬܐ ܒܬܪܝ </syc>
<span class="syc">ܩܐܝܢ</span> | 1635 | 15 | Hebrews 11:4 | <span class="syc">ܒܗܝܡܢܘܬܐ ܩܪܒ ܗܒܝܠ ܕܒܚܬܐ ܕܡܝܬܪܐ ܛܒ ܡܢ ܕܩܐܝܢ ܠܐܠܗܐ ܘܡܛܠܬܗ ܗܘܬ ܥܠܘܗܝ ܣܗܕܘܬܐ ܕܟܐܢܐ ܗܘ ܘܣܗܕ ܥܠ ܩܘܪܒܢܗ ܐܠܗܐ ܘܡܛܠܬܗ ܐܦ ܟܕ ܡܝܝܬ ܡܡܠܠ </syc>
<span class="syc">ܩܘܦܪܘܣ</span> | 1657 | 16 | Acts 4:36 | <span class="syc">ܝܘܣܦ ܕܝܢ ܗܘ ܕܐܬܟܢܝ ܒܪܢܒܐ ܡܢ ܫܠܝܚܐ ܕܡܬܬܪܓܡ ܒܪܐ ܕܒܘܝܐܐ ܠܘܝܐ ܡܢ ܐܬܪܐ ܕܩܘܦܪܣ </syc>
<span class="syc">ܩܠܡܝܣ</span> | 1648 423 | 52 | Philippians 4:3 | <span class="syc">ܐܦ ܡܢܟ ܒܥܐ ܐܢܐ ܒܪ ܙܘܓܝ ܫܪܝܪܐ ܕܬܗܘܐ ܡܥܕܪ ܠܗܝܢ ܕܗܢܝܢ ܠܐܝ ܥܡܝ ܒܐܘܢܓܠܝܘܢ ܥܡ ܩܠܡܝܣ ܘܥܡ ܫܪܟܐ ܕܡܥܕܪܢܝ ܗܢܘܢ ܕܫܡܗܝܗܘܢ ܟܬܝܒܝܢ ܒܟܬܒܐ ܕܚܝܐ </syc>
<span class="syc">ܩܣܪܝܐ</span> | 1641 | 14 | Acts 8:40 | <span class="syc">ܦܝܠܝܦܘܣ ܕܝܢ ܐܫܬܟܚ ܒܐܙܘܛܘܣ ܘܡܢ ܬܡܢ ܡܬܟܪܟ ܗܘܐ ܘܡܣܒܪ ܒܡܕܝܢܬܐ ܟܠܗܝܢ ܥܕܡܐ ܕܐܬܐ ܠܩܣܪܝܐ </syc>
<span class="syc">ܪܘܒܝܠ</span> | 1432 | 5 | Revelation 7:5 | <span class="syc">ܡܢ ܫܪܒܬܗ ܕܝܗܘܕܐ ܬܪܥܣܪ ܐܠܦܝܢ ܡܢ ܫܪܒܬܗ ܕܪܘܒܝܠ ܬܪܥܣܪ ܐܠܦܝܢ ܡܢ ܫܪܒܬܗ ܕܓܕ ܬܪܥܣܪ ܐܠܦܝܢ </syc>
<span class="syc">ܪܚܝܡܐ</span> | 2036 | 8 | Philippians 4:1 | <span class="syc">ܡܟܝܠ ܐܚܝ ܚܒܝܒܐ ܘܪܚܝܡܐ ܘܚܕܘܬܝ ܘܟܠܝܠܝ ܗܟܢܐ ܩܘܡܘ ܒܡܪܢ ܚܒܝܒܝ </syc>
<span class="syc">ܫܘܫܢ</span> | 1168 2092 | 46 | Luke 8:3 | <span class="syc">ܘܝܘܚܢ ܐܢܬܬ ܟܘܙܐ ܪܒܝܬܗ ܕܗܪܘܕܣ ܘܫܘܫܢ ܘܐܚܪܢܝܬܐ ܣܓܝܐܬܐ ܐܝܠܝܢ ܕܡܫܡܫܢ ܗܘܝ ܠܗܘܢ ܡܢ ܩܢܝܢܝܗܝܢ </syc>
<span class="syc">ܫܠܝܡܘܢ</span> | 2060 | 2 | Matthew 1:6 | <span class="syc">ܐܝܫܝ ܐܘܠܕ ܠܕܘܝܕ ܡܠܟܐ ܕܘܝܕ ܐܘܠܕ ܠܫܠܝܡܘܢ ܡܢ ܐܢܬܬܗ ܕܐܘܪܝܐ </syc>
<span class="syc">ܫܡܘܢܐ</span> | 1486 2078 | 5 | Matthew 5:26 | <span class="syc">ܘܐܡܝܢ ܐܡܪ ܐܢܐ ܠܟ ܕܠܐ ܬܦܘܩ ܡܢ ܬܡܢ ܥܕܡܐ ܕܬܬܠ ܫܡܘܢܐ ܐܚܪܝܐ </syc>
<span class="syc">ܫܡܥܘܢ</span> | 1471 1474 1475 2093 2094 2095 2100 986 | 3 | Matthew 4:18 | <span class="syc">ܘܟܕ ܡܗܠܟ ܥܠ ܝܕ ܝܡܐ ܕܓܠܝܠܐ ܚܙܐ ܬܪܝܢ ܐܚܝܢ ܫܡܥܘܢ ܕܐܬܩܪܝ ܟܐܦܐ ܘܐܢܕܪܐܘܣ ܐܚܘܗܝ ܕܪܡܝܢ ܡܨܝܕܬܐ ܒܝܡܐ ܐܝܬܝܗܘܢ ܗܘܘ ܓܝܪ ܨܝܕܐ </syc>
<span class="syc">ܫܡܫܘܢ</span> | 2061 2062 | 2 | Hebrews 11:32 | <span class="syc">ܘܡܢܐ ܬܘܒ ܐܡܪ ܙܥܘܪ ܗܘ ܠܝ ܓܝܪ ܙܒܢܐ ܕܐܫܬܥܐ ܥܠ ܓܕܥܘܢ ܘܥܠ ܒܪܩ ܘܥܠ ܫܡܫܘܢ ܘܥܠ ܢܦܬܚ ܘܥܠ ܕܘܝܕ ܘܥܠ ܫܡܘܐܝܠ ܘܥܠ ܫܪܟܐ ܕܢܒܝܐ </syc>
<span class="syc">ܬܐܘܡܐ</span> | 2123 2125 2126 2127 2586 2772 979 | 5 | Matthew 10:3 | <span class="syc">ܘܦܝܠܝܦܘܣ ܘܒܪ-ܬܘܠܡܝ ܘܬܐܘܡܐ ܘܡܬܝ ܡܟܣܐ ܘܝܥܩܘܒ ܒܪ ܚܠܦܝ ܘܠܒܝ ܕܐܬܟܢܝ ܬܕܝ </syc>
<span class="syc">ܬܐܘܦܝܠܐ</span> | 2812 | 17 | Luke 1:3 | <span class="syc">ܐܬܚܙܝ ܐܦ ܠܝ ܡܛܠ ܕܩܪܝܒ ܗܘܝܬ ܝܨܝܦܐܝܬ ܠܟܠܗܘܢ ܕܟܠ ܡܕܡ ܒܛܟܣܗ ܐܟܬܘܒ ܠܟ ܢܨܝܚܐ ܬܐܘܦܝܠܐ </syc>


### places
lexeme | linked | n-occs | passage | verse text
--- | --- | --- | --- | ---
<span class="syc">ܐܘܪܫܠܡ</span> | 104 | 2 | Matthew 2:1 | <span class="syc">ܟܕ ܕܝܢ ܐܬܝܠܕ ܝܫܘܥ ܒܒܝܬ-ܠܚܡ ܕܝܗܘܕܐ ܒܝܘܡܝ ܗܪܘܕܣ ܡܠܟܐ ܐܬܘ ܡܓܘܫܐ ܡܢ ܡܕܢܚܐ ܠܐܘܪܫܠܡ </syc>
<span class="syc">ܐܠܟܣܢܕܪܝܐ</span> | 572 | 2 | Acts 6:9 | <span class="syc">ܘܩܡܘ ܗܘܘ ܐܢܫܐ ܡܢ ܟܢܘܫܬܐ ܕܡܬܩܪܝܐ ܕܠܝܒܪܛܝܢܘ ܘܩܘܪܝܢܝܐ ܘܐܠܟܣܢܕܪܝܐ ܘܕܡܢ ܩܝܠܝܩܝܐ ܘܡܢ ܐܣܝܐ ܘܕܪܫܝܢ ܗܘܘ ܥܡ ܐܣܛܦܢܘܣ </syc>
<span class="syc">ܐܢܛܝܘܟܝܐ</span> | 10 995 | 44 | Acts 6:5 | <span class="syc">ܘܫܦܪܬ ܗܕܐ ܡܠܬܐ ܩܕܡ ܟܠܗ ܥܡܐ ܘܓܒܘ ܠܐܣܛܦܢܘܣ ܓܒܪܐ ܕܡܠܐ ܗܘܐ ܗܝܡܢܘܬܐ ܘܪܘܚܐ ܕܩܘܕܫܐ ܘܠܦܝܠܝܦܘܣ ܘܠܦܪܟܪܘܣ ܘܠܢܝܩܢܘܪ ܘܠܛܝܡܘܢ ܘܠܦܪܡܢܐ ܘܠܢܝܩܠܐܘܣ ܓܝܘܪܐ ܐܢܛܝܘܟܝܐ </syc>
<span class="syc">ܐܣܦܣ</span> | 288 | 5 | Romans 3:13 | <span class="syc">ܩܒܪܐ ܦܬܝܚܐ ܓܓܪܬܗܘܢ ܘܠܫܢܝܗܘܢ ܢܟܘܠܬܢܝܢ ܘܚܡܬܐ ܕܐܣܦܣ ܬܚܝܬ ܣܦܘܬܗܘܢ </syc>
<span class="syc">ܐܦܣܘܣ</span> | 623 | 69 | Acts 18:19 | <span class="syc">ܘܡܛܝܘ ܠܐܦܣܘܣ ܘܥܠ ܦܘܠܘܣ ܠܟܢܘܫܬܐ ܘܡܡܠܠ ܗܘܐ ܥܡ ܝܗܘܕܝܐ </syc>
<span class="syc">ܐܪܟ</span> | 515 | 1 | Matthew 23:5 | <span class="syc">ܘܟܠܗܘܢ ܥܒܕܝܗܘܢ ܥܒܕܝܢ ܕܢܬܚܙܘܢ ܠܒܢܝ ܐܢܫܐ ܡܦܬܝܢ ܓܝܪ ܬܦܠܝܗܘܢ ܘܡܘܪܟܝܢ ܬܟܠܬܐ ܕܡܪܛܘܛܝܗܘܢ </syc>
<span class="syc">ܓܐܝܘܣ</span> | 1494 | 1 | Acts 19:29 | <span class="syc">ܘܐܫܬܓܫܬ ܟܠܗ ܡܕܝܢܬܐ ܘܪܗܛܘ ܐܟܚܕܐ ܘܐܙܠܘ ܠܬܐܛܪܘܢ ܘܚܛܦܘ ܐܘܒܠܘ ܥܡܗܘܢ ܠܓܐܝܘܣ ܘܠܐܪܣܛܪܟܘܣ ܓܒܪܐ ܡܩܕܘܢܝܐ ܒܢܝ ܠܘܝܬܗ ܕܦܘܠܘܣ </syc>
<span class="syc">ܕܪܐ</span> | 67 | 32 | Matthew 21:44 | <span class="syc">ܘܡܢ ܕܢܦܠ ܥܠ ܟܐܦܐ ܗܕܐ ܢܬܪܥܥ ܘܟܠ ܡܢ ܕܗܝ ܬܦܠ ܥܠܘܗܝ ܬܕܪܝܘܗܝ </syc>
<span class="syc">ܕܪܡܣܘܩ</span> | 66 | 24 | Acts 9:2 | <span class="syc">ܘܫܐܠ ܠܗ ܐܓܪܬܐ ܡܢ ܪܒ ܟܗܢܐ ܕܢܬܠ ܠܗ ܠܕܪܡܣܘܩ ܠܟܢܘܫܬܐ ܕܐܢ ܗܘ ܕܢܫܟܚ ܕܪܕܝܢ ܒܗܕܐ ܐܘܪܚܐ ܓܒܪܐ ܐܘ ܢܫܐ ܢܐܣܘܪ ܢܝܬܐ ܐܢܘܢ ܠܐܘܪܫܠܡ </syc>
<span class="syc">ܚܘܪܐ</span> | 1456 | 4 | Matthew 5:36 | <span class="syc">ܐܦܠܐ ܒܪܫܟ ܬܐܡܐ ܕܠܐ ܡܫܟܚ ܐܢܬ ܠܡܥܒܕ ܒܗ ܡܢܬܐ ܚܕܐ ܕܣܥܪܐ ܐܘܟܡܬܐ ܐܘ ܚܘܪܬܐ </syc>
<span class="syc">ܚܡܬ</span> | 91 | 3 | Matthew 2:16 | <span class="syc">ܗܝܕܝܢ ܗܪܘܕܣ ܟܕ ܚܙܐ ܕܐܬܒܙܚ ܡܢ ܡܓܘܫܐ ܐܬܚܡܬ ܛܒ ܘܫܕܪ ܩܛܠ ܛܠܝܐ ܟܠܗܘܢ ܕܒܝܬ-ܠܚܡ ܘܕܟܠܗܘܢ ܬܚܘܡܝܗ ܡܢ ܒܪ ܬܪܬܝܢ ܫܢܝܢ ܘܠܬܚܬ ܐܝܟ ܙܒܢܐ ܕܥܩܒ ܡܢ ܡܓܘܫܐ </syc>
<span class="syc">ܚܦܪ</span> | 331 | 2 | Matthew 21:33 | <span class="syc">ܫܡܥܘ ܐܚܪܢܐ ܡܬܠܐ ܓܒܪܐ ܚܕ ܐܝܬ ܗܘܐ ܡܪܐ ܒܝܬܐ ܘܢܨܒ ܟܪܡܐ ܘܐܚܕܪܗ ܣܝܓܐ ܘܚܦܪ ܒܗ ܡܥܨܪܬܐ ܘܒܢܐ ܒܗ ܡܓܕܠܐ ܘܐܘܚܕܗ ܠܦܠܚܐ ܘܚܙܩ </syc>
<span class="syc">ܚܪܢ</span> | 216 | 20 | Acts 7:2 | <span class="syc">ܗܘ ܕܝܢ ܐܡܪ ܓܒܪܐ ܐܚܝܢ ܘܐܒܗܬܢ ܫܡܥܘ ܐܠܗܐ ܕܬܫܒܘܚܬܐ ܐܬܚܙܝ ܠܐܒܘܢ ܐܒܪܗܡ ܟܕ ܐܝܬܘܗܝ ܗܘܐ ܒܝܬ-ܢܗܪܝܢ ܥܕ ܠܐ ܢܐܬܐ ܢܥܡܪ ܒܚܪܢ </syc>
<span class="syc">ܛܪܣܘܣ</span> | 196 | 107 | Acts 9:11 | <span class="syc">ܘܡܪܢ ܐܡܪ ܠܗ ܩܘܡ ܙܠ ܠܫܘܩܐ ܕܡܬܩܪܐ ܬܪܝܨܐ ܘܒܥܝ ܒܒܝܬܐ ܕܝܗܘܕܐ ܠܫܐܘܠ ܕܐܝܬܘܗܝ ܡܢ ܛܪܣܘܣ ܡܕܝܢܬܐ ܗܐ ܓܝܪ ܟܕ ܗܘ ܡܨܠܐ </syc>
<span class="syc">ܠܕܝܩܝܐ</span> | 118 | 53 | Colossians 2:1 | <span class="syc">ܨܒܐ ܐܢܐ ܕܝܢ ܕܬܕܥܘܢ ܐܝܢܐ ܐܓܘܢܐ ܐܝܬ ܠܝ ܚܠܦܝܟܘܢ ܘܚܠܦ ܗܢܘܢ ܕܒܠܕܝܩܝܐ ܘܚܠܦ ܫܪܟܐ ܐܝܠܝܢ ܕܦܪܨܘܦܝ ܠܐ ܚܙܘ ܒܒܣܪ </syc>
<span class="syc">ܠܘܕܝܐ</span> | 1526 | 2 | Acts 16:14 | <span class="syc">ܘܐܢܬܬܐ ܚܕܐ ܡܙܒܢܬ ܐܪܓܘܢܐ ܕܕܚܠܐ ܗܘܬ ܡܢ ܐܠܗܐ ܫܡܗ ܗܘܐ ܠܘܕܝܐ ܡܢ ܬܐܘܛܝܪܐ ܡܕܝܢܬܐ ܕܗܕܐ ܦܬܚ ܠܒܗ ܡܪܢ ܘܫܡܥܐ ܗܘܬ ܡܕܡ ܕܐܡܪ ܦܘܠܘܣ </syc>
<span class="syc">ܠܘܩܝܐ</span> | 1533 | 21 | Acts 27:5 | <span class="syc">ܘܥܒܪܢ ܝܡܐ ܕܩܝܠܝܩܝܐ ܘܕܦܡܦܘܠܝܐ ܘܡܛܝܢ ܠܡܘܪܐ ܡܕܝܢܬܐ ܕܠܘܩܝܐ </syc>
<span class="syc">ܡܨܪܝܢ</span> | 7 715 | 1 | Matthew 2:13 | <span class="syc">ܟܕ ܕܝܢ ܐܙܠܘ ܐܬܚܙܝ ܡܠܐܟܐ ܕܡܪܝܐ ܒܚܠܡܐ ܠܝܘܣܦ ܘܐܡܪ ܠܗ ܩܘܡ ܕܒܪ ܠܛܠܝܐ ܘܠܐܡܗ ܘܥܪܘܩ ܠܡܨܪܝܢ ܘܬܡܢ ܗܘܝ ܥܕܡܐ ܕܐܡܪ ܐܢܐ ܠܟ ܥܬܝܕ ܗܘ ܓܝܪ ܗܪܘܕܣ ܠܡܒܥܝܗ ܠܛܠܝܐ ܐܝܟ ܕܢܘܒܕܝܘܗܝ </syc>
<span class="syc">ܡܩܕܘܢܝܐ</span> | 512 | 29 | Acts 16:9 | <span class="syc">ܘܒܚܙܘܐ ܕܠܠܝܐ ܐܬܚܙܝ ܠܦܘܠܘܣ ܐܝܟ ܓܒܪܐ ܚܕ ܡܩܕܘܢܝܐ ܕܩܐܡ ܘܒܥܐ ܡܢܗ ܟܕ ܐܡܪ ܕܬܐ ܠܡܩܕܘܢܝܐ ܘܥܕܪܝܢܝ </syc>
<span class="syc">ܣܘܪܝܐ</span> | 486 | 5 | Matthew 4:24 | <span class="syc">ܘܐܫܬܡܥ ܛܒܗ ܒܟܠܗ ܣܘܪܝܐ ܘܩܪܒܘ ܠܗ ܟܠܗܘܢ ܐܝܠܝܢ ܕܒܝܫ ܒܝܫ ܥܒܝܕܝܢ ܒܟܘܪܗܢܐ ܡܫܚܠܦܐ ܘܐܝܠܝܢ ܕܐܠܝܨܝܢ ܒܬܫܢܝܩܐ ܘܕܝܘܢܐ ܘܕܒܪ ܐܓܪܐ ܘܡܫܪܝܐ ܘܐܣܝ ܐܢܘܢ </syc>
<span class="syc">ܣܟܐ</span> | 1467 | 3 | Matthew 5:34 | <span class="syc">ܐܢܐ ܕܝܢ ܐܡܪ ܐܢܐ ܠܟܘܢ ܠܐ ܬܐܡܘܢ ܣܟ ܠܐ ܒܫܡܝܐ ܕܟܘܪܣܝܐ ܗܘ ܕܐܠܗܐ </syc>
<span class="syc">ܣܪܘܓ</span> | 48 | 2 | Luke 3:35 | <span class="syc">ܒܪ ܣܪܘܓ ܒܪ ܐܪܥܘ ܒܪ ܦܠܓ ܒܪ ܥܒܪ ܒܪ ܫܠܚ </syc>
<span class="syc">ܥܒܐ</span> | 517 | 16 | James 3:5 | <span class="syc">ܗܟܢܐ ܐܦ ܠܫܢܐ ܗܕܡܐ ܗܘ ܙܥܘܪܐ ܘܡܫܬܥܠܐ ܐܦ ܢܘܪܐ ܙܥܘܪܬܐ ܥܒܐ ܣܓܝܐܐ ܡܘܩܕܐ </syc>
<span class="syc">ܥܒܪܐ</span> | 1490 | 5 | Matthew 4:15 | <span class="syc">ܐܪܥܐ ܕܙܒܘܠܘܢ ܐܪܥܐ ܕܢܦܬܠܝ ܐܘܪܚܐ ܕܝܡܐ ܥܒܪܘܗܝ ܕܝܘܪܕܢܢ ܓܠܝܠܐ ܕܥܡܡܐ </syc>
<span class="syc">ܥܙܐ</span> | 87 | 16 | Hebrews 11:37 | <span class="syc">ܐܚܪܢܐ ܐܬܪܓܡܘ ܐܚܪܢܐ ܐܬܢܣܪܘ ܐܚܪܢܐ ܒܦܘܡܐ ܕܣܝܦܐ ܡܝܬܘ ܐܚܪܢܐ ܐܬܟܪܟܘ ܟܕ ܠܒܝܫܝܢ ܡܫܟܐ ܕܐܡܪܐ ܘܕܥܙܐ ܘܣܢܝܩܝܢ ܘܐܠܝܨܝܢ ܘܡܛܪܦܝܢ </syc>
<span class="syc">ܥܩܪ</span> | 25 | 7 | Matthew 13:29 | <span class="syc">ܗܘ ܕܝܢ ܐܡܪ ܠܗܘܢ ܕܠܡܐ ܟܕ ܡܓܒܝܢ ܐܢܬܘܢ ܙܝܙܢܐ ܬܥܩܪܘܢ ܥܡܗܘܢ ܐܦ ܚܛܐ </syc>
<span class="syc">ܦܐܛܪܐ</span> | 157 | 29 | Acts 21:1 | <span class="syc">ܘܦܪܫܢ ܡܢܗܘܢ ܘܪܕܝܢ ܬܪܝܨܐܝܬ ܠܩܘ ܓܙܪܬܐ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܪܘܕܘܣ ܘܡܢ ܬܡܢ ܠܦܐܛܪܐ </syc>
<span class="syc">ܦܪܘܓܝܐ</span> | 1525 | 1 | Acts 2:10 | <span class="syc">ܘܕܡܢ ܐܬܪܐ ܕܦܪܘܓܝܐ ܘܕܦܡܦܘܠܝܐ ܘܕܡܨܪܝܢ ܘܕܐܬܪܘܬܐ ܕܠܘܒܐ ܕܩܪܝܒܝܢ ܠܩܘܪܝܢܐ ܘܐܝܠܝܢ ܕܐܬܘ ܡܢ ܪܗܘܡܐ ܝܗܘܕܝܐ ܘܓܝܘܪܐ </syc>
<span class="syc">ܦܪܬ</span> | 82 | 5 | Acts 1:18 | <span class="syc">ܗܢܘ ܕܩܢܐ ܠܗ ܩܪܝܬܐ ܡܢ ܐܓܪܐ ܕܚܛܝܬܐ ܘܢܦܠ ܥܠ ܐܦܘܗܝ ܥܠ ܐܪܥܐ ܘܐܬܦܪܬ ܡܢ ܡܨܥܬܗ ܘܐܬܐܫܕ ܟܠܗ ܓܘܝܗ </syc>
<span class="syc">ܨܘܪܐ</span> | 4 513 | 8 | Matthew 18:6 | <span class="syc">ܘܟܠ ܕܢܟܫܠ ܠܚܕ ܡܢ ܗܠܝܢ ܙܥܘܪܐ ܕܡܗܝܡܢܝܢ ܒܝ ܦܩܚ ܗܘܐ ܠܗ ܕܬܗܘܐ ܬܠܝܐ ܪܚܝܐ ܕܚܡܪܐ ܒܨܘܪܗ ܘܡܛܒܥ ܒܥܘܡܩܘܗܝ ܕܝܡܐ </syc>
<span class="syc">ܨܠܚ</span> | 2 | 92 | 3_John 1:2 | <span class="syc">ܚܒܝܒܢ ܒܟܠܡܕܡ ܡܨܠܐ ܐܢܐ ܥܠܝܟ ܕܬܨܠܚ ܘܬܗܘܐ ܚܠܝܡ ܐܝܟ ܡܐ ܕܡܨܠܚܐ ܢܦܫܟ </syc>
<span class="syc">ܩܘܦܪܘܣ</span> | 714 | 16 | Acts 4:36 | <span class="syc">ܝܘܣܦ ܕܝܢ ܗܘ ܕܐܬܟܢܝ ܒܪܢܒܐ ܡܢ ܫܠܝܚܐ ܕܡܬܬܪܓܡ ܒܪܐ ܕܒܘܝܐܐ ܠܘܝܐ ܡܢ ܐܬܪܐ ܕܩܘܦܪܣ </syc>
<span class="syc">ܩܝܠܝܩܝܐ</span> | 55 | 115 | Acts 6:9 | <span class="syc">ܘܩܡܘ ܗܘܘ ܐܢܫܐ ܡܢ ܟܢܘܫܬܐ ܕܡܬܩܪܝܐ ܕܠܝܒܪܛܝܢܘ ܘܩܘܪܝܢܝܐ ܘܐܠܟܣܢܕܪܝܐ ܘܕܡܢ ܩܝܠܝܩܝܐ ܘܡܢ ܐܣܝܐ ܘܕܪܫܝܢ ܗܘܘ ܥܡ ܐܣܛܦܢܘܣ </syc>
<span class="syc">ܩܠܘܕܝܐ</span> | 63 | 110 | 2_Timothy 4:21 | <span class="syc">ܢܬܒܛܠ ܠܟ ܕܩܕܡ ܣܬܘܐ ܬܐܬܐ ܫܐܠ ܒܫܠܡܟ ܐܘܒܘܠܘܣ ܘܦܘܕܣ ܘܠܝܢܘܣ ܘܩܠܘܕܝܐ ܘܐܚܐ ܟܠܗܘܢ </syc>
<span class="syc">ܩܣܪܝܐ</span> | 60 | 14 | Acts 8:40 | <span class="syc">ܦܝܠܝܦܘܣ ܕܝܢ ܐܫܬܟܚ ܒܐܙܘܛܘܣ ܘܡܢ ܬܡܢ ܡܬܟܪܟ ܗܘܐ ܘܡܣܒܪ ܒܡܕܝܢܬܐ ܟܠܗܝܢ ܥܕܡܐ ܕܐܬܐ ܠܩܣܪܝܐ </syc>
<span class="syc">ܪܘܕܘܣ</span> | 1534 | 3 | Acts 21:1 | <span class="syc">ܘܦܪܫܢ ܡܢܗܘܢ ܘܪܕܝܢ ܬܪܝܨܐܝܬ ܠܩܘ ܓܙܪܬܐ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܪܘܕܘܣ ܘܡܢ ܬܡܢ ܠܦܐܛܪܐ </syc>
<span class="syc">ܬܠܐ</span> | 200 | 1 | Matthew 18:6 | <span class="syc">ܘܟܠ ܕܢܟܫܠ ܠܚܕ ܡܢ ܗܠܝܢ ܙܥܘܪܐ ܕܡܗܝܡܢܝܢ ܒܝ ܦܩܚ ܗܘܐ ܠܗ ܕܬܗܘܐ ܬܠܝܐ ܪܚܝܐ ܕܚܡܪܐ ܒܨܘܪܗ ܘܡܛܒܥ ܒܥܘܡܩܘܗܝ ܕܝܡܐ </syc>


# Thanks

Dirk Roorda

Data Archiving and Networked Services